## Prep

In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re

## 1. Read the course schedule into a DataFrame

In [2]:
# read and parse schedule.html
with open("schedule.html", 'r', encoding='utf-8') as f:
    data = f.read()
    dom = BeautifulSoup(data)

In [3]:
# scrape data from web and combine them into a dataframe
course_num = []
course_name = []
instructor = []
time = []
rows = dom.select('li.row')
for row in rows:
    item = row.select('span')
    course_num.append(re.search('CSCI.+\d', item[0].text)[0].replace('\u200b', ''))
    course_name.append(item[1].text.strip().replace('  ', '').replace('\n', ''))
    instructor.append(item[2].text.strip().replace('  ', '').replace('\n', ''))
    time.append(item[3].text.strip().replace('  ', '').replace('\n', ''))
combined_info = np.array([course_num, course_name, instructor, time]).T
schedule = pd.DataFrame(combined_info, columns=['Number_Section', 'Name', 'Instructor', 'Time'], index=range(1, len(combined_info)+1))
schedule

,Number_Section,Name,Instructor,Time
1,CSCI-GA.1170-001,Fundamental Algorithms,Chee Yap,T 4:55-6:55PM
2,CSCI-GA.1170-002,Fundamental Algorithms Recitation,Daniel Feldan,R 5:55-6:45PM
3,CSCI-GA.1170-003,Fundamental Algorithms,RichardCole,W 4:55-6:55PM
4,CSCI-GA.1170-004,Fundamental Algorithms Recitation,Zachary DeStefano,F 4:55-5:45PM
5,CSCI-GA.1170-005,Fundamental Algorithms Recitation,Zeming Lin,R 5:55-6:45PM
...,...,...,...,...
156,CSCI-UA.0480-046,Special Topics: Intro to Social Networking,Bhubaneswar Mishra,TR 12:30-1:45PM
157,CSCI-UA.0480-052,Special Topics: Algorithmic Problem Solving,Joanna Klukowska,MW 12:30-1:45PM
158,CSCI-UA.0480-057,Special Topics: Natural Language Processing,Adam Meyers,TR 2:00-3:15PM
159,CSCI-UA.0480-069,Special Topics: Agile Software Development and...,Amos Bloomberg,TR 12:30-1:45PM


* For the course number, I choose to use the one starting with 'CSCI' because it is the format used by the catalog page. As we will eventually merge the two dataframes into one, it's better to use the same format.

In [4]:
# split Number_Section column into Number and Section
schedule['Number'] = [re.search('(.+)-', i)[1] for i in schedule['Number_Section']]
schedule['Section'] = [re.search('-(\d+)', i)[1] for i in schedule['Number_Section']]
schedule

,Number_Section,Name,Instructor,Time,Number,Section
1,CSCI-GA.1170-001,Fundamental Algorithms,Chee Yap,T 4:55-6:55PM,CSCI-GA.1170,001
2,CSCI-GA.1170-002,Fundamental Algorithms Recitation,Daniel Feldan,R 5:55-6:45PM,CSCI-GA.1170,002
3,CSCI-GA.1170-003,Fundamental Algorithms,RichardCole,W 4:55-6:55PM,CSCI-GA.1170,003
4,CSCI-GA.1170-004,Fundamental Algorithms Recitation,Zachary DeStefano,F 4:55-5:45PM,CSCI-GA.1170,004
5,CSCI-GA.1170-005,Fundamental Algorithms Recitation,Zeming Lin,R 5:55-6:45PM,CSCI-GA.1170,005
...,...,...,...,...,...,...
156,CSCI-UA.0480-046,Special Topics: Intro to Social Networking,Bhubaneswar Mishra,TR 12:30-1:45PM,CSCI-UA.0480,046
157,CSCI-UA.0480-052,Special Topics: Algorithmic Problem Solving,Joanna Klukowska,MW 12:30-1:45PM,CSCI-UA.0480,052
158,CSCI-UA.0480-057,Special Topics: Natural Language Processing,Adam Meyers,TR 2:00-3:15PM,CSCI-UA.0480,057
159,CSCI-UA.0480-069,Special Topics: Agile Software Development and...,Amos Bloomberg,TR 12:30-1:45PM,CSCI-UA.0480,069


In [5]:
# show the info of the dataframe
schedule.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160 entries, 1 to 160
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Number_Section  160 non-null    object
 1   Name            160 non-null    object
 2   Instructor      160 non-null    object
 3   Time            160 non-null    object
 4   Number          160 non-null    object
 5   Section         160 non-null    object
dtypes: object(6)
memory usage: 7.6+ KB


In [6]:
# show the first 5 rows
schedule.head(5)

,Number_Section,Name,Instructor,Time,Number,Section
1,CSCI-GA.1170-001,Fundamental Algorithms,Chee Yap,T 4:55-6:55PM,CSCI-GA.1170,001
2,CSCI-GA.1170-002,Fundamental Algorithms Recitation,Daniel Feldan,R 5:55-6:45PM,CSCI-GA.1170,002
3,CSCI-GA.1170-003,Fundamental Algorithms,RichardCole,W 4:55-6:55PM,CSCI-GA.1170,003
4,CSCI-GA.1170-004,Fundamental Algorithms Recitation,Zachary DeStefano,F 4:55-5:45PM,CSCI-GA.1170,004
5,CSCI-GA.1170-005,Fundamental Algorithms Recitation,Zeming Lin,R 5:55-6:45PM,CSCI-GA.1170,005


In [7]:
# show the last 5 rows
schedule.tail(5)

,Number_Section,Name,Instructor,Time,Number,Section
156,CSCI-UA.0480-046,Special Topics: Intro to Social Networking,Bhubaneswar Mishra,TR 12:30-1:45PM,CSCI-UA.0480,046
157,CSCI-UA.0480-052,Special Topics: Algorithmic Problem Solving,Joanna Klukowska,MW 12:30-1:45PM,CSCI-UA.0480,052
158,CSCI-UA.0480-057,Special Topics: Natural Language Processing,Adam Meyers,TR 2:00-3:15PM,CSCI-UA.0480,057
159,CSCI-UA.0480-069,Special Topics: Agile Software Development and...,Amos Bloomberg,TR 12:30-1:45PM,CSCI-UA.0480,069
160,CSCI-UA.0480-071,Special Topics:Collaborating Remotely: Build S...,Evan Korth,TR 3:30-4:45PM,CSCI-UA.0480,071


In [8]:
# random sample 5 rows
schedule.sample(5)

,Number_Section,Name,Instructor,Time,Number,Section
89,CSCI-UA.0004-001,Intro to Web Design & Computer Principles,Alvaro Olsen,MW 8:00-9:15AM,CSCI-UA.0004,001
150,CSCI-UA.0476-001,Processing Big Data for Analytics Applications,Ann Malavet,TR 8:00-9:15AM,CSCI-UA.0476,001
18,CSCI-GA.2271-001,Computer Vision,Rob Fergus,R 7:10-9:10PM,CSCI-GA.2271,001
127,CSCI-UA.0201-021,Computer Systems Organization - Recitation,Jaya Amit Sai Gurrala,T 9:30-10:45AM,CSCI-UA.0201,021
101,CSCI-UA.0101-005,Intro To Computer Science,Duygu Ataman,MW 2:00-3:15PM,CSCI-UA.0101,005


## 2. Read the course catalog into a DataFrame

In [9]:
# read and parse catalog.html
with open("catalog.html", 'r', encoding='utf-8') as f:
    data = f.read()
    dom = BeautifulSoup(data)

In [10]:
# scrape data from web and combine them into a dataframe
number = []
prereqs = []
pts = []
columns = dom.select('li.col-sm-12')
for col in columns:
    item = col.select('p')
    number.append(re.search('[A-Z]+-.+\d', item[0].text.strip())[0])
    pts.append(re.search('(.*)Points', item[1].text.strip())[1].strip())
    prereqs.append(re.search('Prerequisites: (.+[a-zA-Z0-9)])', item[2].text.strip())[1])
combined_cols = np.array([number, prereqs, pts]).T
catalog = pd.DataFrame(combined_cols, columns=['Number', 'Prereqs', 'Points'], index=range(1, len(combined_cols)+1))
catalog

,Number,Prereqs,Points
1,CSCI-GA.1133,None,4
2,CSCI-GA.1144,CSCI-GA 1133 or departmental permission,4
3,CSCI-GA.1170,At least one year of experience with a high-le...,3
4,CSCI-GA.1180,None,3
5,CSCI-GA.2110,Students taking this class should already have...,3
...,...,...,...
92,CSCI-UA.0897,Restricted to declared computer science majors...,1 - 4
93,CSCI-UA.0898,Restricted to declared computer science majors...,1 - 4
94,CSCI-UA.0997,Permission of the department. Does not satisfy...,1 - 4
95,CSCI-UA.0998,Permission of the department. Does not satisfy...,1 - 4


In [11]:
# show the info of the dataframe
catalog.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 1 to 96
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Number   96 non-null     object
 1   Prereqs  96 non-null     object
 2   Points   96 non-null     object
dtypes: object(3)
memory usage: 2.4+ KB


In [12]:
# show the first 5 rows
catalog.head(5)

,Number,Prereqs,Points
1,CSCI-GA.1133,None,4
2,CSCI-GA.1144,CSCI-GA 1133 or departmental permission,4
3,CSCI-GA.1170,At least one year of experience with a high-le...,3
4,CSCI-GA.1180,None,3
5,CSCI-GA.2110,Students taking this class should already have...,3


In [13]:
# show the last 5 rows
catalog.tail(5)

,Number,Prereqs,Points
92,CSCI-UA.0897,Restricted to declared computer science majors...,1 - 4
93,CSCI-UA.0898,Restricted to declared computer science majors...,1 - 4
94,CSCI-UA.0997,Permission of the department. Does not satisfy...,1 - 4
95,CSCI-UA.0998,Permission of the department. Does not satisfy...,1 - 4
96,FRSEM-UA.0597,"Some programming experience in Python, Java, J...",4


In [14]:
# random sample 5 rows
catalog.sample(5)

,Number,Prereqs,Points
11,CSCI-GA.2262,Students must have a working knowledge of fund...,3
81,CSCI-UA.0472,Computer Systems Organization (CSCI-UA 201) an...,4
32,CSCI-GA.2585,"Familiarity with basics in linear algebra, pro...",3
91,CSCI-UA.0521,Permission of the department,4
21,CSCI-GA.2436,None,3


## 3. Put together both DataFrames

In [15]:
# put together schedule and catalog by using pd.merge
merged_course = pd.merge(schedule, catalog, on='Number', how='left')
merged_course.drop(['Number_Section', 'Section'], axis=1, inplace=True)
merged_course = merged_course.iloc[:,[3, 0, 1, 2, 4, 5]]
pd.set_option('display.max_rows', 200)
merged_course

,Number,Name,Instructor,Time,Prereqs,Points
0,CSCI-GA.1170,Fundamental Algorithms,Chee Yap,T 4:55-6:55PM,At least one year of experience with a high-le...,3
1,CSCI-GA.1170,Fundamental Algorithms Recitation,Daniel Feldan,R 5:55-6:45PM,At least one year of experience with a high-le...,3
2,CSCI-GA.1170,Fundamental Algorithms,RichardCole,W 4:55-6:55PM,At least one year of experience with a high-le...,3
3,CSCI-GA.1170,Fundamental Algorithms Recitation,Zachary DeStefano,F 4:55-5:45PM,At least one year of experience with a high-le...,3
4,CSCI-GA.1170,Fundamental Algorithms Recitation,Zeming Lin,R 5:55-6:45PM,At least one year of experience with a high-le...,3
5,CSCI-GA.1170,Fundamental Algorithms Recitation,Nick Lourie,F 4:55-5:45PM,At least one year of experience with a high-le...,3
6,CSCI-GA.1180,Mathematical Techniques For CS Applications,Parijat Dube,M 7:10-9:10PM,None,3
7,CSCI-GA.2110,Programming Languages,Cory Plock,M 4:55-6:55PM,Students taking this class should already have...,3
8,CSCI-GA.2110,Programming Languages Recitation,Yashaswi Shah,R 4:55-5:45PM,Students taking this class should already have...,3
9,CSCI-GA.2110,Programming Languages,Thomas Wies,T 4:55-6:55PM,Students taking this class should already have...,3


## 4. Conclusion

* The dataframe indexes starts from 0 instead of 1. This could be modified by reassigning the indexes (see below).
* There are completely duplicate rows (e.g. row 60-63). This could be modified by df.drop_duplicates (see below).
* I'm not sure whether recitation sections should share the same prereqs and points with the lectures. According to the feature of our dataframes, the recitation sections share the same course numbers with the lectures, therefore sharing the same prereqs and points during the merging process. If we want to specify the difference between lecture and recitation sections, perhaps we need to create a new column that clarify the 'identity' of the course (whether it's a 'Lecture' or a 'Recitation').
* According to the merged dataframe, "how=left" keeps all rows in the first DataFrame. It utilizes and keeps keys from the left dataframe, which is the schedule dataframe in our case.

In [16]:
# drop duplicate rows
merged_course.drop_duplicates(inplace=True)

# reindex the dataframe
merged_course.index = range(1, len(merged_course.index)+1)

merged_course

,Number,Name,Instructor,Time,Prereqs,Points
1,CSCI-GA.1170,Fundamental Algorithms,Chee Yap,T 4:55-6:55PM,At least one year of experience with a high-le...,3
2,CSCI-GA.1170,Fundamental Algorithms Recitation,Daniel Feldan,R 5:55-6:45PM,At least one year of experience with a high-le...,3
3,CSCI-GA.1170,Fundamental Algorithms,RichardCole,W 4:55-6:55PM,At least one year of experience with a high-le...,3
4,CSCI-GA.1170,Fundamental Algorithms Recitation,Zachary DeStefano,F 4:55-5:45PM,At least one year of experience with a high-le...,3
5,CSCI-GA.1170,Fundamental Algorithms Recitation,Zeming Lin,R 5:55-6:45PM,At least one year of experience with a high-le...,3
6,CSCI-GA.1170,Fundamental Algorithms Recitation,Nick Lourie,F 4:55-5:45PM,At least one year of experience with a high-le...,3
7,CSCI-GA.1180,Mathematical Techniques For CS Applications,Parijat Dube,M 7:10-9:10PM,None,3
8,CSCI-GA.2110,Programming Languages,Cory Plock,M 4:55-6:55PM,Students taking this class should already have...,3
9,CSCI-GA.2110,Programming Languages Recitation,Yashaswi Shah,R 4:55-5:45PM,Students taking this class should already have...,3
10,CSCI-GA.2110,Programming Languages,Thomas Wies,T 4:55-6:55PM,Students taking this class should already have...,3
